In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import GradScaler, autocast
from torch.optim import Adam
from torch.nn import MSELoss
from torch import nn, device, save
from torch.optim import Adam, AdamW
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import os
import timeit
import random

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import torch

def prepare_and_save_data(data, dep_var, drop_columns, output_file_path, test_size=0.2, random_state=42):
    # Ensure the output directory exists
    if not os.path.exists(output_file_path):
        os.makedirs(output_file_path, exist_ok=True)

    # Drop specified columns
    if drop_columns:
        data = data.drop(columns=drop_columns)
    
    # Select features and target variable
    X = data.drop(columns=[dep_var])
    y = data[dep_var]

    # Split data into training+validation sets and test set
    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Further split training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)
    X_test_scaled = scaler.transform(X_test)

    # Convert numpy arrays to tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float)
    X_valid_tensor = torch.tensor(X_valid_scaled, dtype=torch.float)
    y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float)

    return X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, X_test_tensor, y_test_tensor

class ANNModel(nn.Module):
    def __init__(self, n_features, num_hidden_layers=2, neurons_per_layer=64, activation_function=F.relu, 
                 dropout_rate=0.1, l1_regularization=0, l2_regularization=0):
        super(ANNModel, self).__init__()

        self.activation_function = activation_function
        self.l1_regularization = l1_regularization
        self.l2_regularization = l2_regularization

        # Build the ANN architecture
        layers = [nn.Linear(n_features, neurons_per_layer), activation_function()]
        for _ in range(num_hidden_layers - 1):
            layers += [
                nn.Linear(neurons_per_layer, neurons_per_layer),
                activation_function(),
                nn.Dropout(dropout_rate)
            ]
        
        # Final layer without activation
        layers.append(nn.Linear(neurons_per_layer, 1))
        
        # Wrap the layers into nn.Sequential
        self.layers = nn.Sequential(*layers)
        
        # Initialize weights
        self.init_weights()

    def init_weights(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

    def regularization_loss(self):
        l1_loss = 0
        l2_loss = 0
        for param in self.parameters():
            l1_loss += torch.norm(param, 1)
            l2_loss += torch.norm(param, 2) ** 2

        return self.l1_regularization * l1_loss + self.l2_regularization * l2_loss

    def forward(self, x):
        for layer in self.layers:
            x = layer(x) if not isinstance(layer, nn.Dropout) else layer(x)
            if any(isinstance(layer, activ) for activ in [nn.ReLU, nn.Tanh, nn.ELU, nn.LeakyReLU, nn.Sigmoid]):
                x = self.activation_function(x)
        return x  
    
def train_ANN_model(model, X_train, y_train, X_valid, y_valid, n_epochs, batch_size=32, learning_rate=1e-3, 
                    patience=10, min_delta=0.0001, max_norm=1.0, num_workers=0, pin_memory=False, 
                    validation_frequency=1, save_directory=None, trial=1):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Convert numpy arrays to PyTorch tensors
    X_train, y_train = torch.tensor(X_train).float().to(device), torch.tensor(y_train).float().to(device)
    X_valid, y_valid = torch.tensor(X_valid).float().to(device), torch.tensor(y_valid).float().to(device)
    
    # Create TensorDataset and DataLoader for both training and validation sets
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)

    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

    optimizer = Adam(model.parameters(), lr=learning_rate)
    criterion = MSELoss()
    scaler = GradScaler()

    best_val_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(n_epochs):
        # Training
        model.train()
        train_losses = []
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            with autocast():
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch.view_as(y_pred))
                # Add regularization
                reg_loss = model.regularization_loss()
                loss += reg_loss
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            scaler.step(optimizer)
            scaler.update()

            train_losses.append(loss.item())

        # Validation
        if epoch % validation_frequency == 0:
            model.eval()
            val_losses = []
            with torch.no_grad():
                for X_batch, y_batch in valid_loader:
                    y_pred = model(X_batch)
                    loss = criterion(y_pred, y_batch.view_as(y_pred))
                    val_losses.append(loss.item())

            avg_val_loss = np.mean(val_losses)
            print(f"Epoch {epoch}: Train Loss = {np.mean(train_losses):.5f}, Val Loss = {avg_val_loss:.5f}")

            if avg_val_loss < best_val_loss - min_delta:
                best_val_loss = avg_val_loss
                early_stopping_counter = 0
                # Save model
                if save_directory:
                    os.makedirs(save_directory, exist_ok=True)
                    save_path = os.path.join(save_directory, f"ANN_model_trial_{trial}_version_{model_ver}.pt")
                else:
                    save_path = f"ANN_model_trial_{trial}.pt"
                save(model.state_dict(), save_path)
            else:
                early_stopping_counter += 1
                if early_stopping_counter >= patience:
                    print(f"Early stopping at epoch {epoch} due to no improvement in validation loss.")
                    break

    return {
        "train_losses": train_losses,
        "val_losses": val_losses,
        "best_val_loss": best_val_loss,
        "early_stopping_counter": early_stopping_counter
    }

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss', linewidth=2)
    plt.plot(val_losses, label='Valid Loss', linewidth=2)
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.legend(fontsize=14)
    plt.title(f'Train and Valid Losses (Trial {trial+1})', fontsize=16)
    plt.grid(True)

    if save_directory:
        os.makedirs(save_directory, exist_ok=True)  # Ensure directory exists
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial+1}.png")
        plt.savefig(save_path)
        print(f"Plot saved to {save_path}")

    plt.show()

def model_save_ANN(X_train, y_train, X_valid, y_valid, n_trials=1, save_directory=None, plot_loss=True):
    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    all_results_params = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")

        start = timeit.default_timer()

        # Randomly generate hyperparameters
        num_hidden_layers = random.choice(range(1, 4))
        neurons_per_layer = random.choice([64, 128, 256])
        dropout_rate = random.choice([0.1, 0.2, 0.3])
        batch_size = random.choice([32, 64, 128])
        learning_rate = random.choice([1e-3, 1e-4, 1e-5])
        activation_function = random.choice([F.relu, F.leaky_relu])
        optimizer_choice = random.choice([Adam, AdamW])
        n_epochs = random.choice(range(50, 101))
        patience = 10

        # # Load data
        # X_train = torch.load(os.path.join(output_file_path, 'X_train.pt'))
        # y_train = torch.load(os.path.join(output_file_path, 'y_train.pt'))
        # X_valid = torch.load(os.path.join(output_file_path, 'X_valid.pt'))
        # y_valid = torch.load(os.path.join(output_file_path, 'y_valid.pt'))

        # Define the ANN model
        model = ANNModel(n_features=X_train.shape[1], num_hidden_layers=num_hidden_layers,
                         neurons_per_layer=neurons_per_layer, activation_function=activation_function,
                         dropout_rate=dropout_rate)

        # Train the ANN model
        training_results = train_ANN_model(model, X_train, y_train, X_valid, y_valid, n_epochs, 
                                           batch_size=batch_size, learning_rate=learning_rate,
                                           patience=patience, optimizer=optimizer_choice, 
                                           save_directory=save_directory, trial=trial)

        train_losses, val_losses = training_results['train_losses'], training_results['val_losses']

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)

        # Collect results and parameters
        params = {
            "num_hidden_layers": num_hidden_layers,
            "neurons_per_layer": neurons_per_layer,
            "dropout_rate": dropout_rate,
            "batch_size": batch_size,
            "learning_rate": learning_rate,
            "optimizer": optimizer_choice.__name__,
            "n_epochs": n_epochs
        }

        results_params = {**params, "trial": trial + 1, "train_loss": train_losses[-1], "val_loss": val_losses[-1]}
        all_results_params.append(results_params)

        if save_directory:
            all_results_params_df = pd.DataFrame(all_results_params)
            all_results_params_df.to_csv(os.path.join(save_directory, "all_results_params.csv"), index=False)

        end = timeit.default_timer()
        print(f"Execution Time for Trial {trial + 1}: {end - start} seconds")

    return all_results_params_df if save_directory else all_results_params

def evaluate_models_and_save_results(X_test, y_test, load_directory=None, save_directory=None, results_file_name='evaluation_results.csv'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    results = []
    file_names = os.listdir(load_directory)
    
    for file_name in file_names:
        model_path = os.path.join(load_directory, file_name)
        loaded_model = torch.load(model_path, map_location=device)
        loaded_model.eval()
        
        # X_test_tensor = torch.load(os.path.join(output_file_path, 'X_valid.pt'))
        # y_test_tensor = torch.load(os.path.join(output_file_path, 'y_valid.pt'))

        with torch.no_grad():
            y_pred_tensor = loaded_model(X_test)
            # Adjust shape if necessary, e.g., y_pred_tensor = y_pred_tensor.squeeze()
        
        y_pred = y_pred_tensor.cpu().numpy()
        y_true = y_test.cpu().numpy()

        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

        results.append({'Model': file_name, 'MAE': mae, 'MSE': mse, 'R2': r2})
    
    results_df = pd.DataFrame(results)
    
    if save_directory:
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)
        results_path = os.path.join(save_directory, results_file_name)
        results_df.to_csv(results_path, index=False)
        print(f"Results saved to {results_path}")
    
    return results_df

def execute_and_evaluate_ANN_models(data, dep_var, drop_columns, n_trials, top_k_models, save_directory, load_directory):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory, exist_ok=True)
        
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_and_save_data(data, dep_var, drop_columns, test_size=0.2, random_state=42)
    
    # Step 1: Train Models and Save Training Results
    model_save_ANN(X_train, y_train, X_valid, y_valid, n_trials=n_trials, save_directory=save_directory, plot_loss=True)
    
    # Step 2: Load and Evaluate Models, then save evaluation results
    evaluate_results_df = evaluate_models_and_save_results(X_test, y_test, load_directory=load_directory, save_directory=save_directory,
                                                           results_file_name='evaluation_results.csv')
    
    # Step 3: Select top k models based on a criterion (e.g., R2 score here for illustration)
    top_models_df = evaluate_results_df.nlargest(top_k_models, 'R2')
    print("Top models based on R2 score:")
    print(top_models_df)
    
    # Save the top models' evaluation results for further inspection
    top_models_df.to_csv(os.path.join(save_directory, "top_models_evaluation.csv"), index=False)
    
    return top_models_df

In [ ]:
n_trials = 10
top_k_models = 3
load_directory = 'models.txt'
save_directory = 'models.txt'
top_models_df = execute_and_evaluate_ANN_models(n_trials=n_trials, top_k_models=top_k_models, save_directory=save_directory)
print(top_models_df)

In [3]:
def evaluate_models_and_save_results(load_directory=None, output_file_path=None, save_directory=None, results_file_name='evaluation_results.csv'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    results = []
    file_names = os.listdir(load_directory)
    
    for file_name in file_names:
        model_path = os.path.join(load_directory, file_name)
        loaded_model = torch.load(model_path, map_location=device)
        loaded_model.eval()
        
        X_test_tensor = torch.load(os.path.join(output_file_path, 'X_valid.pt'))
        y_test_tensor = torch.load(os.path.join(output_file_path, 'y_valid.pt'))

        with torch.no_grad():
            y_pred_tensor = loaded_model(X_test_tensor)
            # Adjust shape if necessary, e.g., y_pred_tensor = y_pred_tensor.squeeze()
        
        y_pred = y_pred_tensor.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()

        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

        results.append({'Model': file_name, 'MAE': mae, 'MSE': mse, 'R2': r2})
    
    results_df = pd.DataFrame(results)
    
    if save_directory:
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)
        results_path = os.path.join(save_directory, results_file_name)
        results_df.to_csv(results_path, index=False)
        print(f"Results saved to {results_path}")
    
    return results_df


In [4]:
def execute_and_evaluate_ANN_models(X_train_path, y_train_path, X_valid_path, y_valid_path, X_test_path, y_test_path,
                                    n_trials, top_k_models, save_directory):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory, exist_ok=True)
    
    # Step 1: Train Models and Save Training Results
    model_save_ANN(n_trials=n_trials, save_directory=save_directory, plot_loss=True,
                   output_file_path=output_file_path)
    
    # Step 2: Load and Evaluate Models, then save evaluation results
    evaluate_results_df = evaluate_models_and_save_results(load_directory=load_directory,
                                                           output_file_path=output_file_path,
                                                           save_directory=save_directory,
                                                           results_file_name='evaluation_results.csv')
    
    # Step 3: Select top k models based on a criterion (e.g., R2 score here for illustration)
    top_models_df = evaluate_results_df.nlargest(top_k_models, 'R2')
    print("Top models based on R2 score:")
    print(top_models_df)
    
    # Save the top models' evaluation results for further inspection
    top_models_df.to_csv(os.path.join(save_directory, "top_models_evaluation.csv"), index=False)
    
    return top_models_df


In [8]:
n_trials = 3
save_directory = 'saved_models'
all_results_params_df = model_save_ANN(n_trials=n_trials, model_save=True, save_directory=save_directory, plot_loss=True, output_file_path=output_file_path)
# Save results_df
all_results_params_df.to_csv(f'{save_directory}_all_results_params_df.csv', index=True)

NameError: name 'output_file_path' is not defined

In [6]:
#  ANN_regression_with_random_search_modify

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

def create_model_regression(n_features, dropout_rate=0.2, activation_function='relu', 
                 optimizer='adam', num_hidden_layers=2, num_neurons_per_layer=64,
                 kernel_regularizer=None, early_stop_patience=None, verbose=None):
    """
    Create a deep learning regression model.

    Args:
    n_features (int): Number of input features.
    dropout_rate (float): Dropout rate.
    activation_function (str): Activation function.
    optimizer (str): Deep learning optimizer.
    num_hidden_layers (int): Number of hidden layers.
    num_neurons_per_layer (int): Number of neurons in hidden layers.
    kernel_regularizer (Optional[Keras Regularizer]): Regularization function for the layers.

    Returns:
    Keras model: A deep learning regression model with the specified architecture.
    """
    model = Sequential()

    model.add(Dense(n_features, activation=activation_function, input_dim=n_features, kernel_regularizer=kernel_regularizer))
    model.add(Dropout(dropout_rate))

    for _ in range(num_hidden_layers):
        model.add(Dense(num_neurons_per_layer, activation=activation_function, kernel_regularizer=kernel_regularizer))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

def ANN_regression_with_random_search(data, dep_var, drop_columns=[], n_iter=50, cv=3,
                                                    verbose=1, dropout_rates=[0.1, 0.2, 0.3, 0.4],
                                                    activation_functions=["relu", "tanh"],
                                                    optimizers=["adam", "rmsprop", "adagrad"],
                                                    batch_sizes=[16, 32, 64, 128], epochs=[50, 100, 200],
                                                    early_stop_patience=[5, 10, 15],
                                                    num_hidden_layers_range=range(1, 4),
                                                    neurons_per_layer_range=[16, 32, 48, 64],
                                                    learning_rate_range=[0.001, 0.01, 0.1],
                                                    model_save=True, save_directory=None, plot_loss=True,
                                                    test_size=0.2, random_state=42, new_data=None):
    """
    Runs a deep learning regression process with Randomized Search for hyperparameter optimization.

    Args:
    Required:
    data (pd.DataFrame): Data to be used for model training.
    dep_var (str): Target variable.

    Optional:
    drop_columns (List[str]): Columns to be dropped from dataframe.
    n_iter (int): Number of iterations for RandomizedSearchCV.
    cv (int): Number of cross validation folds.
    verbose (int): Print log level.
    activation_functions (List[str]): Activation functions to be used.
    dropout_rates (List[float]): Dropout rates.
    optimizers (List[str]): Deep learning optimizers to be used.
    batch_sizes (List[int]): Batch sizes to be used.
    epochs (List[int]): Number of epochs to be used.
    early_stop_patience (List[int]): Early stopping patience values.
    num_hidden_layers_range (List[int]): Number of hidden layer values to search.
    neurons_per_layer_range (List[int]): Number of neurons per layer values.
    learning_rate_range (List[float]): Learning rates.
    model_save (bool): If True, save the model.
    save_directory (Optional[str]): Path to the directory to save the model.
    test_size (float): Test set proportion.
    random_state (int): Random state for train_test_split.
    new_data (pd.DataFrame): If provided, predict target values for this data.

    Returns:
    pd.DataFrame: A dataframe with model results.
    """
    # drop columns
    X = data.drop([dep_var] + drop_columns, axis=1).values
    y = data[dep_var].values
    # split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    # scale data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    # get number of features
    n_features = X_train_scaled.shape[1]
    # create model
    deep_learning_model = KerasRegressor(model=create_model_regression, model__n_features=n_features, verbose=verbose)
    
    # Define the random search parameters
    param_dist = {
        "model__activation_function": activation_functions,
        "model__dropout_rate": dropout_rates,
        "model__optimizer": optimizers,
        "model__num_hidden_layers": num_hidden_layers_range,
        "model__num_neurons_per_layer": neurons_per_layer_range,
        "model__kernel_regularizer": [L1L2(l1=0, l2=regularization_strength) for regularization_strength in learning_rate_range],
        "batch_size": batch_sizes,
        "epochs": epochs,
        "model__early_stop_patience": early_stop_patience
    }
    
    # Create the RandomizedSearchCV object
    random_search = RandomizedSearchCV(deep_learning_model, param_distributions=param_dist, n_iter=n_iter,
                                       cv=cv, error_score=np.nan, verbose=verbose)
    
    # Create EarlyStopping and ModelCheckpoint callbacks
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=verbose, patience=early_stop_patience)
    mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=verbose, save_best_only=True)
    
    # Fit to the training data
    random_search.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),callbacks=[es, mc])
    
    # Get the best model from the RandomizedSearchCV
    best_model = random_search.best_estimator_
    
    # Get the best parameters from the RandomizedSearchCV
    best_params = random_search.best_params_
    
    # create and fit final model with best parameters
    final_model = create_model_regression(n_features=n_features, dropout_rate=best_params['model__dropout_rate'],
                                           activation_function=best_params['model__activation_function'],
                                           optimizer=best_params['model__optimizer'],
                                           num_hidden_layers=best_params['model__num_hidden_layers'],
                                           num_neurons_per_layer=best_params['model__num_neurons_per_layer'],
                                           kernel_regularizer=best_params['model__kernel_regularizer'],
                                           early_stop_patience=best_params['model__early_stop_patience'])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=verbose, patience=best_params['model__early_stop_patience'])
    mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=verbose, save_best_only=True)
    history = final_model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                              epochs=best_params['epochs'], batch_size=best_params['batch_size'], callbacks=[es, mc])
    
    # Plotting the training and validation loss
    if plot_loss:
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Loss Plot')
        plt.show()

    # Save the model
    if model_save:
        model_save_path = os.path.join(save_directory, 'best_regression_ANN_model_RandomSearch.h5') if save_directory else 'best_regression_ANN_model_RandomSearch.h5'
        final_model.save(model_save_path)

    # Predict target values for training and test data
    y_pred_train = final_model.predict(X_train_scaled)
    y_pred_test = final_model.predict(X_test_scaled)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # Concatenate X_train_scaled and X_test_scaled back into X_scaled
    X_scaled = np.concatenate((X_train_scaled, X_test_scaled), axis=0)

    # Concatenate y_train and y_test back into a y
    y = np.concatenate((y_train, y_test), axis=0)
    
    # Wrap the Keras model in a scikit-learn estimator
    estimator = KerasRegressor(model=final_model, epochs=50, batch_size=32, verbose=0)

    # Calculate CV RMSE and its standard deviation using the cross_val_score for the best model
    cv_scores = cross_val_score(estimator, X_scaled, y, scoring='neg_mean_squared_error', cv=5)
    rmse_scores = np.sqrt(-cv_scores)
    cv_rmse = np.mean(rmse_scores)
    cv_rmse_std = np.std(rmse_scores)
    
    # Predict values for new data
    if new_data is not None:
        new_X = new_data.drop([dep_var] + drop_columns, axis=1).values
        new_y = new_data[dep_var].values
        new_X_scaled = scaler.fit_transform(new_X)
        new_y_pred = final_model.predict(new_X_scaled)
        new_y_residual = new_y - new_y_pred.flatten()
    else:
        new_y_pred = None
        new_y_residual = None
    
    # Create a dataframe with results
    results = {
        "Model": ["ANN Regression with Random Search (Best Model)"],
        "Train R²": [round(train_r2, 3)],
        "Test R²": [round(test_r2, 3)],
        "Train RMSE": [round(train_rmse, 3)],
        "Test RMSE": [round(test_rmse, 3)],
        "CV RMSE": [round(cv_rmse, 3)],
        "CV RMSE Std": [round(cv_rmse_std, 3)],
        "Best Hyperparameters": [best_params],
        "New Predicted": [new_y_pred] if new_y_pred is not None else [None],
        "New Residual": [new_y_residual] if new_y_residual is not None else [None]
    }

    return pd.DataFrame(results)

2024-04-07 23:10:25.280227: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 23:10:25.282307: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 23:10:25.302332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 23:10:25.806519: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
n_trials = 3
save_directory = 'saved_models'
all_results_params_df = model_save_ANN(n_trials=n_trials, model_save=True, save_directory=save_directory, plot_loss=True, output_file_path=output_file_path, model_ver=model_ver)
# Save results_df
all_results_params_df.to_csv(f'{save_directory}_all_results_params_df.csv', index=True)


NameError: name 'output_file_path' is not defined